In [1]:
import pickle
with open('../Data/allplayerGameLogs.pickle', 'rb') as handle:
  allplayerGameLogs = pickle.load(handle)

In [5]:
import pandas as pd

In [2]:
allplayerGameLogs.head()

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22015,203112,0021500889,"FEB 29, 2016",SAC vs. OKC,L,28,5,9,0.556,...,0,4,2,3,0,0,3,10,-16,1
1,22015,203112,0021500867,"FEB 26, 2016",SAC vs. LAC,L,15,1,1,1.000,...,0,0,0,0,1,0,3,4,-12,1
2,22015,203112,0021500854,"FEB 24, 2016",SAC vs. SAS,L,23,6,9,0.667,...,2,2,0,2,0,1,4,15,13,1
3,22015,203112,0021500842,"FEB 23, 2016",SAC @ DEN,W,13,0,2,0.000,...,3,3,0,0,3,0,1,0,-1,1
4,22015,203112,0021500816,"FEB 19, 2016",SAC vs. DEN,W,15,0,0,0.000,...,3,3,0,0,0,1,1,0,2,1


In [4]:
allplayerGameLogs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18964 entries, 0 to 42
Data columns (total 27 columns):
SEASON_ID          18964 non-null object
Player_ID          18964 non-null float64
Game_ID            18964 non-null object
GAME_DATE          18964 non-null object
MATCHUP            18964 non-null object
WL                 18964 non-null object
MIN                18964 non-null float64
FGM                18964 non-null float64
FGA                18964 non-null float64
FG_PCT             18964 non-null float64
FG3M               18964 non-null float64
FG3A               18964 non-null float64
FG3_PCT            18964 non-null float64
FTM                18964 non-null float64
FTA                18964 non-null float64
FT_PCT             18964 non-null float64
OREB               18964 non-null float64
DREB               18964 non-null float64
REB                18964 non-null float64
AST                18964 non-null float64
STL                18964 non-null float64
BLK              

Modify the datetime

In [7]:
allplayerGameLogs['GAME_DATE'] = pd.to_datetime(allplayerGameLogs['GAME_DATE'])

In [8]:
allplayerGameLogs['GAME_DATE'].head()

0   2016-02-29
1   2016-02-26
2   2016-02-24
3   2016-02-23
4   2016-02-19
Name: GAME_DATE, dtype: datetime64[ns]

In [9]:
del allplayerGameLogs['VIDEO_AVAILABLE']

In [10]:
allplayerGameLogs.columns

Index([u'SEASON_ID', u'Player_ID', u'Game_ID', u'GAME_DATE', u'MATCHUP', u'WL',
       u'MIN', u'FGM', u'FGA', u'FG_PCT', u'FG3M', u'FG3A', u'FG3_PCT', u'FTM',
       u'FTA', u'FT_PCT', u'OREB', u'DREB', u'REB', u'AST', u'STL', u'BLK',
       u'TOV', u'PF', u'PTS', u'PLUS_MINUS'],
      dtype='object')

Add columns to indicate whether the player has double-double or triple-double.

In [69]:
dd = ((allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['AST']>=10))| \
     ((allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['REB']>=10))| \
     ((allplayerGameLogs['AST']>=10) & (allplayerGameLogs['REB']>=10))| \
     ((allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['STL']>=10))| \
     ((allplayerGameLogs['REB']>=10) & (allplayerGameLogs['BLK']>=10))| \
     ((allplayerGameLogs['STL']>=10) & (allplayerGameLogs['BLK']>=10))| \
     ((allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['BLK']>=10))

In [70]:
sum(dd)

1517

In [71]:
allplayerGameLogs['DouBL']= dd

In [74]:
ttt = (allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['AST']>=10) & (allplayerGameLogs['REB']>=10)| \
      (allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['STL']>=10) & (allplayerGameLogs['REB']>=10)| \
      (allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['BLK']>=10) & (allplayerGameLogs['REB']>=10)| \
      (allplayerGameLogs['AST']>=10) & (allplayerGameLogs['BLK']>=10) & (allplayerGameLogs['REB']>=10)| \
      (allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['STL']>=10) & (allplayerGameLogs['AST']>=10)| \
      (allplayerGameLogs['STL']>=10) & (allplayerGameLogs['AST']>=10) & (allplayerGameLogs['REB']>=10)

In [75]:
sum(ttt)

47

In [76]:
allplayerGameLogs['TriBL']= ttt

calculate the fantasy point for a player to add a column in the data frame

In [93]:
allplayerGameLogs['FanPTs'] = 3.5 * allplayerGameLogs['FG3M'] + 1*allplayerGameLogs['FTM'] \
+ 2*(allplayerGameLogs['FGM']-allplayerGameLogs['FG3M']) \
+ 1.25*allplayerGameLogs['REB']+1.5*allplayerGameLogs['AST'] \
+ 2*allplayerGameLogs['STL']+ 2*allplayerGameLogs['BLK'] \
+ 1.5*allplayerGameLogs['DouBL'] + 3*allplayerGameLogs['TriBL'] \
- 0.5*allplayerGameLogs['TOV']

Test case to verify

In [96]:
davis = allplayerGameLogs[allplayerGameLogs['Player_ID']==203076]

In [97]:
davis[['GAME_DATE','MATCHUP','PTS','AST','REB','BLK','STL','TOV','DouBL','TriBL','FanPTs']]

,GAME_DATE,MATCHUP,PTS,AST,REB,BLK,STL,TOV,DouBL,TriBL,FanPTs
0,2016-02-25,NOP vs. OKC,30,2,6,4,1,2,False,False,50.50
1,2016-02-23,NOP @ WAS,9,3,20,1,2,3,False,False,43.00
2,2016-02-21,NOP @ DET,59,4,20,1,0,2,True,False,93.50
3,2016-02-19,NOP vs. PHI,34,1,7,2,2,2,False,False,51.25
4,2016-02-11,NOP @ OKC,23,2,3,0,2,1,False,False,33.25
5,2016-02-10,NOP vs. UTA,19,5,4,1,0,0,False,False,34.00
6,2016-02-08,NOP @ MIN,27,1,8,0,2,0,False,False,43.00
7,2016-02-06,NOP @ CLE,24,0,11,2,0,2,True,False,42.25
8,2016-02-04,NOP vs. LAL,39,0,11,1,1,1,True,False,57.75
9,2016-02-03,NOP @ SAS,28,4,10,4,4,4,True,False,62.00


Already checked.. it does match the official record from my playing history.

In [98]:
with open('../Data/allplayerFantasyGameLogs.pickle', 'wb') as handle:
  pickle.dump(allplayerGameLogs,handle)

In [ ]:
# function to aggregate statistics for each player given a game date

In [ ]:
# function to generate feature tables, namely training and testing data sets given a game date, where training data set contains all 
# information before the predefined game date

In [ ]:
# function to generate a score(PTS) leader board given a game date

In [ ]:
# function to generate a rebound(REB) leader board given a game date

In [ ]:
# function to generate a rebound(FG3_PCT) leader board given a game date

In [ ]:
# function to generate a assist(AST) leader board given a game date

In [ ]:
# function to generate a block(BLK) leader board given a game date

In [ ]:
# function to generate a turnover(TOV) leader board given a game date

In [ ]:
# function to generate a efficiency(PLUS_MINUS) leader board given a game date

In [ ]:
# function to identify the favorable matchup given a player by a game date